In [2]:
import pandas as pd
import numpy as np
import warnings

# Scikit-learn 라이브러리
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# TensorFlow / Keras 라이브러리
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# 경고 메시지 비활성화
warnings.filterwarnings('ignore', category=FutureWarning)
tf.get_logger().setLevel('ERROR')

# --- 1. 데이터 준비 ---
try:
    df10 = pd.read_csv('pm10.csv')
except FileNotFoundError:
    print("pm10.csv 파일을 찾을 수 없습니다. 스크립트와 동일한 디렉토리에 있는지 확인하세요.")
    exit()

features = ['PM2.5', '오 존', '이산화질소', '일산화탄소', '아황산가스',
            '평균기온(°C)', '평균 풍속(m/s)', '평균 상대습도(%)', '평균 현지기압(hPa)', 'PM10_MA7', 'PM10_MA30', 'PM10lag',
            'Autumn', 'Spring', 'Summer', 'Winter']

X_raw, y = [], []
for i in range(7, len(df10)):
    X_raw.append(df10.loc[i-7:i-1, features].values)
    y.append(df10.loc[i, 'PM10'])

X_raw = np.array(X_raw)
y = np.array(y)

# --- 2. 최초 데이터 분할 (Train / Final Test) ---
# 전체 데이터의 80%를 훈련+검증 세트로, 20%를 최종 테스트 세트로 분할합니다.
split_ratio = 0.8
split_index = int(len(X_raw) * split_ratio)

X_train_val, y_train_val = X_raw[:split_index], y[:split_index]
X_final_test, y_final_test = X_raw[split_index:], y[split_index:]

print(f"전체 데이터: {len(X_raw)}개")
print(f"훈련+검증 세트 (80%): {len(X_train_val)}개")
print(f"최종 테스트 세트 (20%): {len(X_final_test)}개")
print("-" * 50)


# --- 3. 모델 및 평가 함수 정의 ---
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(45, activation='tanh', return_sequences=True, input_shape=input_shape))
    model.add(LSTM(22, activation='tanh'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse')
    return model

def evaluate(y_true, y_pred):
    return {
        'R2': r2_score(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred)
    }

# --- 4. 훈련 세트 내 중첩 교차 검증 (모델 선택 단계) ---
n_runs = 5  # 시간 단축을 위해 RUN 횟수 조정
n_splits = 5
results_svr, results_lstm = [], []

print(f"=== 훈련 세트 내 SVR vs LSTM 교차 검증 시작 (N_Runs={n_runs}, K-Folds={n_splits}) ===")
for i in range(n_runs):
    print(f"\n>> RUN {i+1}/{n_runs} 시작 <<")
    fold_scores_svr, fold_scores_lstm = [], []
    # 훈련+검증 세트 내에서만 TimeSeriesSplit 수행
    tscv = TimeSeriesSplit(n_splits=n_splits)
    for fold, (train_index, val_index) in enumerate(tscv.split(X_train_val)):
        print(f"  - Fold {fold+1}/{n_splits} 진행 중...")
        X_train, X_val = X_train_val[train_index], X_train_val[val_index]
        y_train, y_val = y_train_val[train_index], y_train_val[val_index]

        # SVR
        X_train_svr, X_val_svr = X_train.reshape(len(X_train), -1), X_val.reshape(len(X_val), -1)
        svr_model = SVR().fit(X_train_svr, y_train)
        y_pred_svr = svr_model.predict(X_val_svr)
        fold_scores_svr.append(evaluate(y_val, y_pred_svr))

        # LSTM
        input_shape = (X_train.shape[1], X_train.shape[2])
        lstm_model = create_lstm_model(input_shape)
        lstm_es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        lstm_model.fit(X_train, y_train, validation_data=(X_val, y_val),
                       epochs=200, callbacks=[lstm_es], batch_size=64, verbose=0)
        y_pred_lstm = lstm_model.predict(X_val, verbose=0).flatten()
        fold_scores_lstm.append(evaluate(y_val, y_pred_lstm))

    results_svr.append(pd.DataFrame(fold_scores_svr[1:]).mean().to_dict())
    results_lstm.append(pd.DataFrame(fold_scores_lstm[1:]).mean().to_dict())

# --- 5. 교차 검증 결과 분석 및 최종 모델 선택 ---
df_results_svr = pd.DataFrame(results_svr)
df_results_lstm = pd.DataFrame(results_lstm)

mean_r2_svr = df_results_svr['R2'].mean()
mean_r2_lstm = df_results_lstm['R2'].mean()

print("\n\n" + "="*50)
print(">>> 교차 검증 평균 성능 (모델 선택용) <<<")
print(f"SVR  평균 R2: {mean_r2_svr:.4f}")
print(f"LSTM 평균 R2: {mean_r2_lstm:.4f}")

# R2 점수가 더 높은 모델을 최종 모델로 선택
best_model_name = 'LSTM' if mean_r2_lstm > mean_r2_svr else 'SVR'
print(f"\n>> 최종 모델로 '{best_model_name}' 선택 <<")
print("="*50)


# --- 6. 최종 모델 훈련 및 평가 ---
print("\n=== 최종 모델 훈련 및 평가 시작 ===")
# 전체 훈련+검증 세트로 최종 모델을 다시 학습
if best_model_name == 'LSTM':
    print("LSTM 모델을 전체 훈련 데이터로 재학습합니다...")
    final_model = create_lstm_model((X_train_val.shape[1], X_train_val.shape[2]))
    # 최종 테스트셋에 대한 EarlyStopping은 의미가 없으므로, 최적 epoch를 찾기 위해 전체 데이터로 학습
    final_model.fit(X_train_val, y_train_val, epochs=80, batch_size=64, verbose=0) # Epoch는 이전 CV에서 수렴한 평균 epoch 정도로 설정

    # 최종 테스트 세트로 성능 평가
    y_final_pred = final_model.predict(X_final_test, verbose=0).flatten()
else:
    print("SVR 모델을 전체 훈련 데이터로 재학습합니다...")
    X_train_val_svr = X_train_val.reshape(len(X_train_val), -1)
    X_final_test_svr = X_final_test.reshape(len(X_final_test), -1)

    final_model = SVR()
    final_model.fit(X_train_val_svr, y_train_val)

    # 최종 테스트 세트로 성능 평가
    y_final_pred = final_model.predict(X_final_test_svr)

final_performance = evaluate(y_final_test, y_final_pred)

print("\n\n" + "="*50)
print(f">>> 최종 모델의 성능 (Final Test Set) <<<")
print("="*50)
print(f"  - R2:   {final_performance['R2']:.4f}")
print(f"  - RMSE: {final_performance['RMSE']:.2f}")
print(f"  - MAE:  {final_performance['MAE']:.2f}")
print("="*50)

전체 데이터: 1820개
훈련+검증 세트 (80%): 1456개
최종 테스트 세트 (20%): 364개
--------------------------------------------------
=== 훈련 세트 내 SVR vs LSTM 교차 검증 시작 (N_Runs=5, K-Folds=5) ===

>> RUN 1/5 시작 <<
  - Fold 1/5 진행 중...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


  - Fold 2/5 진행 중...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


  - Fold 3/5 진행 중...


...

  - Fold 5/5 진행 중...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)




>>> 교차 검증 평균 성능 (모델 선택용) <<<
SVR  평균 R2: 0.1941
LSTM 평균 R2: 0.2427

>> 최종 모델로 'LSTM' 선택 <<

=== 최종 모델 훈련 및 평가 시작 ===
LSTM 모델을 전체 훈련 데이터로 재학습합니다...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)




>>> 최종 모델의 성능 (Final Test Set) <<<
  - R2:   0.4480
  - RMSE: 0.09
  - MAE:  0.07


In [3]:
import pandas as pd
import numpy as np
import warnings

# Scikit-learn 라이브러리
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# TensorFlow / Keras 라이브러리
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

# 경고 메시지 비활성화
warnings.filterwarnings('ignore', category=FutureWarning)
tf.get_logger().setLevel('ERROR')

# --- 1. 데이터 준비 ---
try:
    df10 = pd.read_csv('pm25.csv')
except FileNotFoundError:
    print("pm25.csv 파일을 찾을 수 없습니다. 스크립트와 동일한 디렉토리에 있는지 확인하세요.")
    exit()

features = ['PM10', '오 존', '이산화질소', '일산화탄소', '아황산가스',
            '평균기온(°C)', '평균 풍속(m/s)', '평균 상대습도(%)', '평균 현지기압(hPa)', 'PM2.5_MA7', 'PM2.5_MA30', 'PM2.5lag',
            'Autumn', 'Spring', 'Summer', 'Winter']

X_raw, y = [], []
for i in range(7, len(df10)):
    X_raw.append(df10.loc[i-7:i-1, features].values)
    y.append(df10.loc[i, 'PM2.5'])

X_raw = np.array(X_raw)
y = np.array(y)

# --- 2. 최초 데이터 분할 (Train / Final Test) ---
# 전체 데이터의 80%를 훈련+검증 세트로, 20%를 최종 테스트 세트로 분할합니다.
split_ratio = 0.8
split_index = int(len(X_raw) * split_ratio)

X_train_val, y_train_val = X_raw[:split_index], y[:split_index]
X_final_test, y_final_test = X_raw[split_index:], y[split_index:]

print(f"전체 데이터: {len(X_raw)}개")
print(f"훈련+검증 세트 (80%): {len(X_train_val)}개")
print(f"최종 테스트 세트 (20%): {len(X_final_test)}개")
print("-" * 50)


# --- 3. 모델 및 평가 함수 정의 ---
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(45, activation='tanh', return_sequences=True, input_shape=input_shape))
    model.add(LSTM(22, activation='tanh'))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse')
    return model

def evaluate(y_true, y_pred):
    return {
        'R2': r2_score(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAE': mean_absolute_error(y_true, y_pred)
    }

# --- 4. 훈련 세트 내 중첩 교차 검증 (모델 선택 단계) ---
n_runs = 5  # 시간 단축을 위해 RUN 횟수 조정
n_splits = 5
results_svr, results_lstm = [], []

print(f"=== 훈련 세트 내 SVR vs LSTM 교차 검증 시작 (N_Runs={n_runs}, K-Folds={n_splits}) ===")
for i in range(n_runs):
    print(f"\n>> RUN {i+1}/{n_runs} 시작 <<")
    fold_scores_svr, fold_scores_lstm = [], []
    # 훈련+검증 세트 내에서만 TimeSeriesSplit 수행
    tscv = TimeSeriesSplit(n_splits=n_splits)
    for fold, (train_index, val_index) in enumerate(tscv.split(X_train_val)):
        print(f"  - Fold {fold+1}/{n_splits} 진행 중...")
        X_train, X_val = X_train_val[train_index], X_train_val[val_index]
        y_train, y_val = y_train_val[train_index], y_train_val[val_index]

        # SVR
        X_train_svr, X_val_svr = X_train.reshape(len(X_train), -1), X_val.reshape(len(X_val), -1)
        svr_model = SVR().fit(X_train_svr, y_train)
        y_pred_svr = svr_model.predict(X_val_svr)
        fold_scores_svr.append(evaluate(y_val, y_pred_svr))

        # LSTM
        input_shape = (X_train.shape[1], X_train.shape[2])
        lstm_model = create_lstm_model(input_shape)
        lstm_es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        lstm_model.fit(X_train, y_train, validation_data=(X_val, y_val),
                       epochs=200, callbacks=[lstm_es], batch_size=64, verbose=0)
        y_pred_lstm = lstm_model.predict(X_val, verbose=0).flatten()
        fold_scores_lstm.append(evaluate(y_val, y_pred_lstm))

    results_svr.append(pd.DataFrame(fold_scores_svr[1:]).mean().to_dict())
    results_lstm.append(pd.DataFrame(fold_scores_lstm[1:]).mean().to_dict())

# --- 5. 교차 검증 결과 분석 및 최종 모델 선택 ---
df_results_svr = pd.DataFrame(results_svr)
df_results_lstm = pd.DataFrame(results_lstm)

mean_r2_svr = df_results_svr['R2'].mean()
mean_r2_lstm = df_results_lstm['R2'].mean()

print("\n\n" + "="*50)
print(">>> 교차 검증 평균 성능 (모델 선택용) <<<")
print(f"SVR  평균 R2: {mean_r2_svr:.4f}")
print(f"LSTM 평균 R2: {mean_r2_lstm:.4f}")

# R2 점수가 더 높은 모델을 최종 모델로 선택
best_model_name = 'LSTM' if mean_r2_lstm > mean_r2_svr else 'SVR'
print(f"\n>> 최종 모델로 '{best_model_name}' 선택 <<")
print("="*50)


# --- 6. 최종 모델 훈련 및 평가 ---
print("\n=== 최종 모델 훈련 및 평가 시작 ===")
# 전체 훈련+검증 세트로 최종 모델을 다시 학습
if best_model_name == 'LSTM':
    print("LSTM 모델을 전체 훈련 데이터로 재학습합니다...")
    final_model = create_lstm_model((X_train_val.shape[1], X_train_val.shape[2]))
    # 최종 테스트셋에 대한 EarlyStopping은 의미가 없으므로, 최적 epoch를 찾기 위해 전체 데이터로 학습
    final_model.fit(X_train_val, y_train_val, epochs=80, batch_size=64, verbose=0) # Epoch는 이전 CV에서 수렴한 평균 epoch 정도로 설정

    # 최종 테스트 세트로 성능 평가
    y_final_pred = final_model.predict(X_final_test, verbose=0).flatten()
else:
    print("SVR 모델을 전체 훈련 데이터로 재학습합니다...")
    X_train_val_svr = X_train_val.reshape(len(X_train_val), -1)
    X_final_test_svr = X_final_test.reshape(len(X_final_test), -1)

    final_model = SVR()
    final_model.fit(X_train_val_svr, y_train_val)

    # 최종 테스트 세트로 성능 평가
    y_final_pred = final_model.predict(X_final_test_svr)

final_performance = evaluate(y_final_test, y_final_pred)

print("\n\n" + "="*50)
print(f">>> 최종 모델의 성능 (Final Test Set) <<<")
print("="*50)
print(f"  - R2:   {final_performance['R2']:.4f}")
print(f"  - RMSE: {final_performance['RMSE']:.2f}")
print(f"  - MAE:  {final_performance['MAE']:.2f}")
print("="*50)

전체 데이터: 1820개
훈련+검증 세트 (80%): 1456개
최종 테스트 세트 (20%): 364개
--------------------------------------------------
=== 훈련 세트 내 SVR vs LSTM 교차 검증 시작 (N_Runs=5, K-Folds=5) ===

>> RUN 1/5 시작 <<
  - Fold 1/5 진행 중...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


  - Fold 2/5 진행 중...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


  - Fold 3/5 진행 중...


...

  - Fold 5/5 진행 중...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)




>>> 교차 검증 평균 성능 (모델 선택용) <<<
SVR  평균 R2: 0.2856
LSTM 평균 R2: 0.3004

>> 최종 모델로 'LSTM' 선택 <<

=== 최종 모델 훈련 및 평가 시작 ===
LSTM 모델을 전체 훈련 데이터로 재학습합니다...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)




>>> 최종 모델의 성능 (Final Test Set) <<<
  - R2:   0.4664
  - RMSE: 0.12
  - MAE:  0.10
